In [ ]:
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from pandas import DataFrame
import numpy as np
import math
import heapq
from tqdm import tqdm
import random

In [ ]:
#!pip install --upgrade tensorflow
#!pip install --upgrade pandas
#!pip install --upgrade numpy
tf.__version__

'2.4.1'

In [ ]:
df = pd.read_csv('users-feeds.csv')

In [ ]:
def shrink_users_df(df,user_id):
    userIds = np.random.choice(df[user_id].unique(),
                                    size=int(len(df[user_id].unique())*0.20),
                                    replace=False)
    return df.loc[df[user_id].isin(userIds)]
def add_negative_samples(df, item_tag, user_tag,label_tag):

    updated_df = pd.DataFrame(columns=[user_tag,item_tag,label_tag])
    all_feeds = df[item_tag].unique()
    users, items, labels = [], [], []

    user_item_set = set(zip(df[user_tag], df[item_tag]))
    num_negatives = 2

    for (u, i) in user_item_set:
        users.append(u)
        items.append(i)
        labels.append(1) # items that the user has interacted with are positive
        for _ in range(num_negatives):
            # randomly select an item
            negative_item = np.random.choice(all_feeds)
            # check that the user has not interacted with this item
            while (u, negative_item) in user_item_set:
                negative_item = np.random.choice(all_feeds)
            users.append(u)
            items.append(negative_item)
            labels.append(0) # items not interacted with are negative
    updated_df[user_tag] = users
    updated_df[item_tag] = items
    updated_df[label_tag] = labels
    del df
    return updated_df

def mask_first(x):
    """
    Return a list of 0 for the first item and 1 for all others
    """
    result = np.ones_like(x)
    result[0] = 0

    return result
# needs to add validate in the future
def train_test_split(full_df):
    df_test = df.copy(deep=True)
    df_test = df_test.groupby(['user']).first()

    df_test['user'] = df_test.index
    df_test = df_test[['user', 'feed_id','is_following_feed']]
    df_test = df_test.rename_axis(None, axis=1)

    df_train = df.copy(deep=True)
    mask = df.groupby(['user'])['user'].transform(mask_first).astype(bool)

    df_train = df.loc[mask]
    return df_train, df_test
def random_mini_batches(U, I, L, mini_batch_size=256):
  
    """Returns a list of shuffeled mini batched of a given size.
    Args:
        U (list): All users for every interaction 
        I (list): All items for every interaction
        L (list): All labels for every interaction.
    
    Returns:
        mini_batches (list): A list of minibatches containing sets
            of batch users, batch items and batch labels 
            [(u, i, l), (u, i, l) ...]
    """

    mini_batches = []

    shuffled_U, shuffled_I, shuffled_L = U, I, L

    num_complete_batches = int(math.floor(len(U)/mini_batch_size))
    for k in range(0, num_complete_batches):
        mini_batch_U = shuffled_U[k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_I = shuffled_I[k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_L = shuffled_L[k * mini_batch_size : k * mini_batch_size + mini_batch_size]

        mini_batch = (mini_batch_U, mini_batch_I, mini_batch_L)
        mini_batches.append(mini_batch)

    if len(U) % mini_batch_size != 0:
        mini_batch_U = shuffled_U[num_complete_batches * mini_batch_size: len(U)]
        mini_batch_I = shuffled_I[num_complete_batches * mini_batch_size: len(U)]
        mini_batch_L = shuffled_L[num_complete_batches * mini_batch_size: len(U)]

        mini_batch = (mini_batch_U, mini_batch_I, mini_batch_L)
        mini_batches.append(mini_batch)

    return mini_batches

In [ ]:
df = shrink_users_df(df, 'user')


df.loc[:, 'is_following_feed'] = 1

df = add_negative_samples(df,'feed_id','user','is_following_feed')

print(df.sample(10))

           user  feed_id  is_following_feed
7920059  334144  6551764                  0
160615   300055  1869024                  0
2623230   39847   211372                  1
6362694  322887  5784488                  1
606539    79995  1449535                  0
1450443  173939  3873432                  1
1249312  112903   546234                  0
7089800   44146  4901438                  0
1208339   86468  5270818                  0
6761850   12130    52219                  1


In [ ]:
df_train, df_test = train_test_split(df)


In [ ]:
# HYPERPARAMS
#-------------

num_neg = 2
latent_features = 8
epochs = 20
batch_size = 256
learning_rate = 0.002


#-------------------------
# TENSORFLOW GRAPH
#-------------------------

graph = tf.Graph()

with graph.as_default():

    # Define input placeholders for user, item and label.
    user = tf.placeholder(tf.int32, shape=(None, 1))
    item = tf.placeholder(tf.int32, shape=(None, 1))
    label = tf.placeholder(tf.int32, shape=(None, 1))

    # User embedding for MLP
    mlp_u_var = tf.Variable(tf.random_normal([len(df['user'].unique()), 32], stddev=0.05),
            name='mlp_user_embedding')
    mlp_user_embedding = tf.nn.embedding_lookup(mlp_u_var, user)

    # Item embedding for MLP
    mlp_i_var = tf.Variable(tf.random_normal([len(df['feed_id'].unique()), 32], stddev=0.05),
            name='mlp_item_embedding')
    mlp_item_embedding = tf.nn.embedding_lookup(mlp_i_var, item)

    # User embedding for GMF
    gmf_u_var = tf.Variable(tf.random_normal([len(df['user'].unique()), latent_features],
        stddev=0.05), name='gmf_user_embedding')
    gmf_user_embedding = tf.nn.embedding_lookup(gmf_u_var, user)

    # Item embedding for GMF
    gmf_i_var = tf.Variable(tf.random_normal([len(df['feed_id'].unique()), latent_features],
        stddev=0.05), name='gmf_item_embedding')
    gmf_item_embedding = tf.nn.embedding_lookup(gmf_i_var, item)

    # Our GMF layers
    gmf_user_embed = tf.keras.layers.Flatten()(gmf_user_embedding)
    gmf_item_embed = tf.keras.layers.Flatten()(gmf_item_embedding)
    gmf_matrix = tf.multiply(gmf_user_embed, gmf_item_embed)

    # Our MLP layers
    mlp_user_embed = tf.keras.layers.Flatten()(mlp_user_embedding)
    mlp_item_embed = tf.keras.layers.Flatten()(mlp_item_embedding)
    mlp_concat = tf.keras.layers.concatenate([mlp_user_embed, mlp_item_embed])

    mlp_dropout = tf.keras.layers.Dropout(0.2)(mlp_concat)

    mlp_layer_1 = tf.keras.layers.Dense(64, activation='relu', name='layer1')(mlp_dropout)
    mlp_batch_norm1 = tf.keras.layers.BatchNormalization(name='batch_norm1')(mlp_layer_1)
    mlp_dropout1 = tf.keras.layers.Dropout(0.2, name='dropout1')(mlp_batch_norm1)

    mlp_layer_2 = tf.keras.layers.Dense(32, activation='relu', name='layer2')(mlp_dropout1)
    mlp_batch_norm2 = tf.keras.layers.BatchNormalization(name='batch_norm1')(mlp_layer_2)
    mlp_dropout2 = tf.keras.layers.Dropout(0.2, name='dropout1')(mlp_batch_norm2)

    mlp_layer_3 = tf.keras.layers.Dense(16, activation='relu', name='layer3')(mlp_dropout2)
    mlp_layer_4 = tf.keras.layers.Dense(8, activation='relu', name='layer4')(mlp_layer_3)

    # We merge the two networks together
    merged_vector = tf.keras.layers.concatenate([gmf_matrix, mlp_layer_4])

    # Our final single neuron output layer. 
    output_layer = tf.keras.layers.Dense(1,
            kernel_initializer="lecun_uniform",
            name='output_layer')(merged_vector)

    # Our loss function as a binary cross entropy. 
    loss = tf.losses.sigmoid_cross_entropy(label, output_layer)

    # Train using the Adam optimizer to minimize our loss.
    opt = tf.train.AdamOptimizer(learning_rate = learning_rate)
    step = opt.minimize(loss)

    # Initialize all tensorflow variables.
    init = tf.global_variables_initializer()


session = tf.Session(config=None, graph=graph)
session.run(init)

In [ ]:
for epoch in range(epochs):

    # Get our training input.
    user_input, item_input, labels = df_train['user'], df_train['feed_id'], df_train['is_following_feed']

    # Generate a list of minibatches.
    minibatches = random_mini_batches(user_input, item_input, labels)

    # This has noting to do with tensorflow but gives
    # us a nice progress bar for the training
    progress = tqdm(total=len(minibatches))

    # Loop over each batch and feed our users, items and labels
    # into our graph. 
    for minibatch in minibatches:
        feed_dict = {user: np.array(minibatch[0]).reshape(-1,1),
                    item: np.array(minibatch[1]).reshape(-1,1),
                    label: np.array(minibatch[2]).reshape(-1,1)}
   
        # Execute the graph.
        _, l = session.run([step, loss], feed_dict)

        # Update the progress
        progress.update(1)
        progress.set_description('Epoch: %d - Loss: %.3f' % (epoch+1, l))

    progress.close()

Streaming output truncated to the last 5000 lines.
Epoch: 1 - Loss: 0.516:  13%|█▎        | 4491/34435 [01:12<08:21, 59.67it/s]